# Training the UNET

## Import

In [1]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np
import cv2
from keras.models import Model
from keras.layers import Input, Dense
import keras.losses as losses

### 전처리

In [2]:
import cv2

n_train_B = 400
# n_train_A = 10
n_test = 20

train_green = []

for i in range (0,n_train_B,3):
    filename = "E:/Project_Ch00cy/CGVR/TextureSynthesis/deep-textures-main/testingTextures/capture/draw-" + str(i) + ".bmp"

    try:
        image = cv2.imread(filename, cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # BGR->RGB
        image = cv2.resize(image,(256,256), interpolation=cv2.INTER_AREA)
    except Exception as e:
        print(str(e))
    image = np.array(image)
    train_green.append(image)
    
train_green = np.array(train_green, dtype="float32")
train_green /= 255.0

In [3]:
train_purple = []

for i in range (3,n_train_B+3,3):
    filename = "E:/Project_Ch00cy/CGVR/TextureSynthesis/deep-textures-main/testingTextures/capture/draw-" + str(i) + ".bmp"
    try:
        image = cv2.imread(filename, cv2.IMREAD_COLOR)
        image = cv2.resize(image,(256,256), interpolation=cv2.INTER_AREA)
    except Exception as e:
        print(str(e)+str(i))
    image = np.array(image)
    train_purple.append(image)
    
train_purple = np.array(train_purple, dtype="float32")
train_purple /= 255.0

### 모델 생성

In [4]:
input_g = tf.keras.Input(shape=(256,256,3))
        
conv0 = layers.Conv2D(16, (3,3), activation='relu', padding = 'same')(input_g)
mp0 = layers.MaxPooling2D((2,2))(conv0)
        
conv1 = layers.Conv2D(32, (3,3), activation='relu', padding = 'same')(mp0)
mp1 = layers.MaxPooling2D((2,2))(conv1)

conv2 = layers.Conv2D(64, (3,3), activation='relu', padding = 'same')(mp1)
mp2 = layers.MaxPooling2D((2,2))(conv2)
        
conv3 = layers.Conv2D(128, (3,3), activation='relu', padding = 'same')(mp2)
mp3 = layers.MaxPooling2D((2,2))(conv3)
        
conv4 = layers.Conv2D(256, (3,3), activation='relu', padding = 'same')(mp3)
mp4 = layers.MaxPooling2D((2,2))(conv4)
        
output_e = layers.Conv2D(512, (3,3), activation='relu', padding = 'same')(mp4)

convt1 = layers.Conv2DTranspose(256, (3,3), activation='relu', padding='same')(output_e) #512아니고 256
upsamp1 = layers.UpSampling2D((2,2))(convt1)
skipcon1 = layers.Concatenate(axis=3)([conv4, upsamp1])
conv6 = layers.Conv2D(256, (3,3), activation = 'relu', padding='same')(skipcon1)

convt2 = layers.Conv2DTranspose(128, (3,3), activation='relu', padding='same')(conv6)
upsamp2 = layers.UpSampling2D((2,2))(convt2)
skipcon2 = layers.Concatenate(axis=3)([conv3, upsamp2])
conv7 = layers.Conv2D(128, (3,3), activation = 'relu', padding='same')(skipcon2)

convt3 = layers.Conv2DTranspose(64, (3,3), activation='relu', padding='same')(conv7)
upsamp3 = layers.UpSampling2D((2,2))(convt3)
skipcon3 = layers.Concatenate(axis=3)([conv2, upsamp3])
conv8 = layers.Conv2D(64, (3,3), activation='relu', padding='same')(skipcon3)

convt4 = layers.Conv2DTranspose(32, (3,3), activation='relu', padding='same')(conv8)
upsamp4 = layers.UpSampling2D((2,2))(convt4)
skipcon4 = layers.Concatenate(axis=3)([conv1, upsamp4])
conv9 = layers.Conv2D(32, (3,3), activation='relu', padding='same')(skipcon4)
        
convt5 = layers.Conv2DTranspose(16, (3,3), activation='relu', padding='same')(conv9)
upsamp5 = layers.UpSampling2D((2,2))(convt5)
skipcon5 = layers.Concatenate(axis=3)([conv0, upsamp5])
conv10 = layers.Conv2D(16, (3,3), activation='relu', padding='same')(skipcon5)

output_p = layers.Conv2DTranspose(3, (3,3), activation='relu', padding='same')(conv10)

### 학습

In [5]:
model = Model(inputs=input_g, outputs=output_p)
model.compile(optimizer='adam', loss=losses.MeanSquaredError())
model.fit(train_green, train_purple, validation_split=0.2, epochs=100, verbose=2)

Epoch 1/100
4/4 - 9s - loss: 0.0274 - val_loss: 0.0171 - 9s/epoch - 2s/step
Epoch 2/100
4/4 - 7s - loss: 0.0130 - val_loss: 0.0096 - 7s/epoch - 2s/step
Epoch 3/100
4/4 - 7s - loss: 0.0086 - val_loss: 0.0095 - 7s/epoch - 2s/step
Epoch 4/100
4/4 - 7s - loss: 0.0079 - val_loss: 0.0086 - 7s/epoch - 2s/step
Epoch 5/100
4/4 - 6s - loss: 0.0074 - val_loss: 0.0088 - 6s/epoch - 2s/step
Epoch 6/100
4/4 - 7s - loss: 0.0072 - val_loss: 0.0081 - 7s/epoch - 2s/step
Epoch 7/100
4/4 - 7s - loss: 0.0068 - val_loss: 0.0078 - 7s/epoch - 2s/step
Epoch 8/100
4/4 - 6s - loss: 0.0064 - val_loss: 0.0071 - 6s/epoch - 2s/step
Epoch 9/100
4/4 - 6s - loss: 0.0057 - val_loss: 0.0065 - 6s/epoch - 2s/step
Epoch 10/100
4/4 - 7s - loss: 0.0052 - val_loss: 0.0059 - 7s/epoch - 2s/step
Epoch 11/100
4/4 - 7s - loss: 0.0046 - val_loss: 0.0052 - 7s/epoch - 2s/step
Epoch 12/100
4/4 - 7s - loss: 0.0042 - val_loss: 0.0048 - 7s/epoch - 2s/step
Epoch 13/100
4/4 - 6s - loss: 0.0038 - val_loss: 0.0044 - 6s/epoch - 2s/step
Epoch 14

### 테스트

In [6]:
test_green = []

for i in range (n_train_B+2,n_train_B+n_test+1,3):
    filename = "E:/Project_Ch00cy/CGVR/TextureSynthesis/deep-textures-main/testingTextures/capture/draw-" + str(i) + ".bmp"

    try:
        image = cv2.imread(filename, cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (256,256), interpolation=cv2.INTER_AREA)
    except Exception as e:
        print(str(e)+str(i))
    image = np.array(image)
    test_green.append(image)

test_green = np.array(test_green, dtype="float32")
test_green /= 255.0

### 이미지 결과 확인

In [7]:
def generate_images(test_input, test_output):
    plt.figure(figsize=(10, 10))
    display_list = [test_input, test_output]
    title = ['Input Image', 'Predicted Image']
    
    for i in range(2):
        plt.subplot(1, 2, i+1)
        plt.title(title[i])
        plt.imshow(display_list[i])
        plt.axis('off')
    plt.show()

In [8]:
test_prediction = model(test_green)
len(test_prediction)

7

In [9]:
plt.imshow(test_prediction[0])
plt.show()

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


: 

In [ ]:
generate_images(test_prediction[1],test_prediction[2])

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


: 

In [ ]:
plt.imshow((test_prediction[1] * 255).astype(np.uint8))

AttributeError: EagerTensor object has no attribute 'astype'. 
        If you are looking for numpy-related methods, please run the following:
        from tensorflow.python.ops.numpy_ops import np_config
        np_config.enable_numpy_behavior()
      

In [ ]:
test_prediction = model(test_green)

i = 0
for image in test_prediction:
    generate_images(test_green[i], image)
    i += 1

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


: 